### The Promise Monad (container)

In [1]:
import pymonad

In [4]:
from pymonad.promise import Promise, _Promise

In [5]:
Promise.insert(1)

In [6]:
await Promise.insert(1)

1

### Map Method

In [7]:
import asyncio

async def wait_and_then_add_one(x: int) -> int:
    print(f"before waiting {x} sec...")
    await asyncio.sleep(x)
    print(f"after waiting {x} sec...")
    return x+1

In [8]:
def add_one(x: int) -> int:
    return x + 1

In [9]:
p = Promise.insert(1).map(wait_and_then_add_one).map(add_one)

In [10]:
p

In [11]:
await Promise.insert(1).map(wait_and_then_add_one).map(wait_and_then_add_one)

before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


3

### Bind method

In [12]:
def generate_new_promise(x: int) -> _Promise[int]:
    print("generating a new promise")
    return Promise.insert(x).map(wait_and_then_add_one)

In [13]:
p = Promise.insert(1).bind(generate_new_promise)

In [14]:
await p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...


2

### Then method 

In [15]:
await Promise.insert(1).then(generate_new_promise).then(wait_and_then_add_one)

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


3

### Catching Exception

In [16]:
def raise_exception_when_too_large(upper_bound: int):
    def wrapper(value: int):
        if value > upper_bound:
            raise ValueError(f"Too large. Value must be lower than {upper_bound}")
        return value
    return wrapper

In [17]:
max_limit = 2

In [18]:
p = Promise.insert(1)\
        .then(generate_new_promise)\
        .then(wait_and_then_add_one)\
        .then(raise_exception_when_too_large(2))

In [19]:
await p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
before waiting 2 sec...
after waiting 2 sec...


ValueError: Too large. Value must be lower than 2

In [20]:
def resetting_to_upper_bound(upper_bound: int):
    def wrapper(e: Exception):
        print(f"Raised Exception: {e} \n Falling back to {upper_bound}")
        return upper_bound
    return wrapper

In [21]:
safe_p = p = Promise.insert(1)\
        .then(generate_new_promise)\
        .then(raise_exception_when_too_large(1))\
        .then(wait_and_then_add_one)\
        .catch(resetting_to_upper_bound(max_limit))

In [22]:
await safe_p

generating a new promise
before waiting 1 sec...
after waiting 1 sec...
Raised Exception: Too large. Value must be lower than 1 
 Falling back to 2


2

### Compose Promises

In [24]:
from pymonad.promise import async_func

@async_func
def my_func(x: int, y: int = 1, z: int = 1):
    return (x + 2*y)/z

In [29]:
async def fakeComputation(x):
    print(f"Computation with {x} started")
    await asyncio.sleep(x)
    print(f"Computation with {x} done")
    return x

In [34]:
x = Promise.insert(1).map(fakeComputation)

y = Promise.insert(2).map(fakeComputation)

z = Promise.insert(3).map(fakeComputation)

promiseSum = my_func(x, y=1, z=z)

final_result = promiseSum.map(fakeComputation)

In [35]:
await final_result

Computation with 1 started
Computation with 3 started
Computation with 1 done
Computation with 3 done
Computation with 1.0 started
Computation with 1.0 done


1.0